## Hacer cambios en modelos
makemigrations genera el archivo para la migración
migrate implementa los cambios

In [ ]:
python manage.py makemigrations
python manage.py migrate

## Borrar Migraciones y Compresión de Migraciones
Migration_Number es la cantidad de migraciones que vamos a agrupar

In [ ]:
python manage.py squashmigrations <App_Label> <Migration_Number>

In [ ]:
python manage.py squashmigrations ecommerce 0003

### Guardando datos usando el shell de Python

In [ ]:
python manage.py shell

In [ ]:
from ecommerce.models import ProductModel
ProductModel.objects.create(title="from shell", price=999)
ProductModel.objects.create(title="second from shell", price=999, description="testing creation from shell")

### Queryset es la lista que contiene todos los objetos almacenados para el modelo especificado

In [ ]:
queryset = ProductModel.objects.all()
qs = ProductModel.objects.al()

### objects.get
### save()
### delete()

## Validación de Campos en los Modelos
### ecommerce/validators.py

In [ ]:
from django.core.exceptions import ValidationError
BLOCKED_WORDS = [
    "barato",
    "malo"
]

def validate_blocked_words(value):
    init_string = f"{value}".lower()
    unique_words = set(init_string.split())
    blocked_words = set(BLOCKED_WORDS)
    invalid_words = (unique_words & blocked_words)
    has_error = len(invalid_words > 0)
    if has_error:
        errors = []
         for i, invalid_word in enumerate(invalid_words):
            msg = "{} es una palabara no permitida".format(invalid_word)
            errors.append(msg)
        raise ValidationError(errors)
    return value

### ecommerce/models.py

In [ ]:
from django.db import models

from .validators import validate_blocked_words

class ProductModel(models.Model):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField()

    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        super().save(*args, **kwargs)

### Cómo agregar opciones a los campos de nuestros modelos

In [ ]:
from django.db import models

from .validators import validate__blocked_words

# [(VALOR_EN_DB, VALOR_PARA_USUARIO)]
PUBLISH_STATE_CHOICES = [
    ("BR", "BORRADOR"),
    ("PU", "PUBLICO"),
    ("PR", "PRIVADO"),
]

class ProductModel(models.Model):
    state = models.CharField(max_length=2, choices=PUBLISH_STATE_CHOICES, default="BR")

    def is_published(self):
        return self.state == "PU"

### Cómo agregar opcions avanzadas a los campos de nuestros modelos

In [ ]:
from django.db import models

from .validators import validate_blocked_words

class ProductModel(models.Model):
    class ProductStateOptions(models.TextChoices):
        PUBLISHED = "PU", "PUBLICO"
        DRAFT = "BR", "BORRADOR"
        PRIVATE = "PR", "PRIVADO"

    state = models.CharField(max_length=2, choices=ProductsStateOptions.choices, default=ProductStateOptions.DRAFT)

    def is_published(self):
        return self.state == ProductsStateOptions.PUBLISEHD

## Modelo Abstracto como base

### base/models.py

In [ ]:
from django.db import models
from django.utils import timezone

class BasePublishModel(models.Model):
    class PublishStateOptions(models.TextChoices):
        PUBLISHED = "PU", "PUBLICO"
        DRAFT = "BR", "BORRADOR"
        PRIVATE = "PR", "PRIVADO"
    
    state = models.CharField(max_length=2, choices=PublishStateOptions.choices, default=PublishStateOptions.DRAFT)
    timestamp = models.DateTimeField(auto_now_add=True)
    updated = models.DateTimeField(auto_now_add=True)
    publish_timestamp = models.DateTimeField(auto_now_add=False, auto_now=False, null=True)

    class Meta:
        abstract = True
        ordering = ["-updated", "-timestamp"]

    @property
    def state_is_published(self):
        return self.state == self.PublishStateOptions.PUBLISHED

    def save(self, *args, **kwargs):
        if self.state_is_published and self.publish_timestamp is None:
            self.publish_timestamp = timezone.now()
        else:
            self.publish_timestamp = None
        super(*args, **kwargs)

    def is_published(self):
        publish_timestamp = self.publish_timestamp
        return self.state_is_published and publish_timestamp < timezone.now()

### ecommerce/models.py

In [ ]:
from django.db import models

from base.models import BasePublishModel
from .validators import validate_blocked_words

class ProductModel(BasePublishModel):
    title = models.TextField(default="")
    description = models.TextField(default="")
    seller = models.TextField(default="")
    color = models.TextField(default="")
    price = models.FloatField(default=0.0)
    product_dimensions = models.FloatField(default=0.0)

    def save(self, *args, **kwargs):
        validate_blocked_words(self.title)
        super().save(*args, **kwargs)

### config/settings.py

In [ ]:
# Application definitions
INSTALLED_APPS = [
    "pages.apps.PagesConfig",
    "ecommerce.apps.EcommerceConfig", # <-----
    "base.apps.BaseConfig", # <-----
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

In [ ]:
python manage.py makemigrations
python manage.py migrate

## Creación a granel

In [ ]:
products_data =  []
for i in range(1,100):
    new_data={"title": "Producto {}".format(i), "price": i*100+99.99}
    products_data.append(new_data)

In [ ]:
from ecommerce.models import ProductModel
new_inventory = []
for product_data in products_data:
    new_inventory.append(ProductModel(**product_data))

In [ ]:
ProductModel.objects.bulk_create(new_inventory, ignore_conflicts=True)

## SlugField y Señales en Modelos
- slug: se usa para los urls:
- title: escritorio con altura ajustable
- slug: /escritorio-con-altura-ajustable
- url: www.mywebsite.com/escritorio-con-altura-ajustable
- url: www.mywebsite.com/1

### Signals / Señales
- pre_save
- post_save
- pre_delete
- post_delete
- pre_init
- post_init
- pre_migrate
- post_migrate

In [ ]:
from django.db.models import signals
dir(signals)

In [ ]:
from django.db import models
from django.db.models.signals import pre_save
from django.utils.text import slugify

from base.models import BasePublishModel
from .validators import validate_blocked_words

class ProductModel(BasePublishModel)
    ...
    slug = models.SlugField(null=True, blank=True, db_index=True)

    def get_absolute_url(self):
        return f"/product/{self.slug}"

def slugify_pre_save(sender, instance, *args, **kwargs):
    if instance.slug is None or instance.slug == "":
        new_slug = slugify(instance.title)
        MyModel = instance.__class__
        qs = MyModel.objects.filter(slug__startswith=new_slug).exclude(id=instance.id)
        if qs.count() == 0:
            instance.slug = new_slug
        else:
            instance.slug = f"{new_slug}-{qs.count()}"

pre_save.connect(slugify_pre_save, sender=ProductModel)

## Fixtures para almacenar y cargar data

In [ ]:
python manage.py dumpdata ecommerce --indent 4 --format json

In [ ]:
python manage.py dumpdata ecommerce --indent 4 --format json > ecommerce/fixtures/ProductModel.json

In [ ]:
python manage.py loaddata ecommerce/fixtures/ProductModel.json

## Llaves foráneas en modelos
### ecommerce/models.py

In [ ]:
...
from django.conf import settings

User = settings.AUTH_USER_MODEL

class ProductModel(BasePublishModel):
    ...
    user = models.ForeignKey(User, null=True, on_delete=models.SET_NULL)